In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import tinygen

In [139]:
from dotenv import load_dotenv
import os

_ = load_dotenv()

# Mock Tools

In [ ]:
from tinygen.tool import Tool

find_tool = Tool(
    "find",
    """find PATTERN
returns a list of paths that match given
PATTERN. PATTERN is a valid regular expression for Python's
re.compile.""",
    lambda arg: input(f'find({arg}): '),
)

grep_tool = Tool(
    "grep",
    """grep PATTERN returns a list of paths to files containing given
PATTERN. PATTERN is a valid regular expression for Python's
re.compile.""",
    lambda arg: input(f'grep({arg}): '),
)

open_tool = Tool(
    "open",
    """open PATH
returns contents of file at PATH.""",
    lambda arg: input(f'open({arg}): '),
)

write_tool = Tool(
    "write",
    """write PATH CONTENT
write CONTENT to the file at PATH. This overwrites existing content if any.""",
    lambda arg: input(f'write({arg}): '),
)

tools = [find_tool, grep_tool, open_tool, write_tool]

# Planning

- Plan + Self-Reflection: https://platform.openai.com/playground/p/jxYEE69LEHREHqf9e7I6LgGj


In [ ]:
from tinygen.plan import plan

task = 'The programming task: Add a new FastAPI endpoint for "/api/diff_me" that accepts githubUrl and prompt as query parameters'
the_plan = plan(tools, task)

In [ ]:
print(the_plan)

# Plan Execution
- Execute (React): https://platform.openai.com/playground/p/BZp876JtEj3ss8SLqI2Acs85

In [ ]:
from tinygen.tool import describe_tools
import openai

def create_agent_msgs(tools: List[Tool], task: str, plan: str) -> List[dict]:
    system_prompt = """You exist in an environment where you can interact with the file system using distinct tools below:
    
{tool_descriptions}

Given a programming task and a step-by-step sequence of actions using the tools described above, the objective is to complete the task.

Each assistant response should be formatted as follows:

OBSERVATION: <Observation about preceding user message in context of accomplishing the programming task>
THOUGHT: <Thought on next step to accomplish programming task, following the step-by-step sequence of the plan.
ACTION: <Specifies one of above tool to use with arguments to pass to tools>

When the task is completed, the ACTION should be "ACTION: DONE"

Examples:

OBSERVATION: The user provided a programming task to "Add Hello World to the bottom of the README file", with the plan to find the file, open the file, then writing to file.
THOUGHT: First, I should find the README file.
ACTION: find "README"

RESULT: "./README.md"

OBSERVATION: There is a file named "./README.md" at path "./README.md"
THOUGHT: Next, I should open the file I found.
ACTION: open "./README.md"

RESULT: "# README\nThis project is used as an example file"

OBSERVATION: The README.md file contains the text above.
THOUGHT: I should append to the file using the write tool using the existing contents and "Hello World" string
ACTION: write "./README.md" "# README\nThis project is used as an example file\nHello World"

RESULT: OK

OBSERVATION: The README.md file was updated.
THOUGHT: I have updated the README.md file. The task is complete.
ACTION: DONE
    """.format(tool_descriptions=describe_tools(tools))

    user_msg = """The Task: 
{task}

The Plan is:
{plan}
    """.format(task=task, plan=plan)

    return [
        { 'role': 'system', 'content': system_prompt },
        { 'role': 'user', 'content': user_msg },
    ]
    

In [ ]:
import re
ACTION_PAT = re.compile('ACTION: (.*)$')

@dataclass
class Action:
    name: str
    args: [str]

def extract_action(msg: dict) -> Optional[Action]:
    match = ACTION_PAT.search(msg['content'])
    if not match:
        return None
    components = match[1].split()
    name = components[0].strip()
    args = components[1:] if len(components) > 1 else []
    return Action(name, list(map(str.strip, args)))
    

In [ ]:
import re
from typing import Callable

def print_message(msg: dict):
    print('*** {role}:\n{content}\n'.format(role = msg['role'].upper(), content = msg['content']))

def resolve_tool(tools: List[Tool], action: Action) -> Optional[Callable[[], str]]:
    if not action:
        return None
    tool = next((t for t in tools if action.name == t.name), None)
    if not tool:
        return None
    return lambda : tool.call(action.args)


In [ ]:
class Agent:
    
    def __init__(self, tools: List[Tool], task: str, plan: str, debug=False):
        self.tools = tools
        self.plan = plan
        self.messages = create_agent_msgs(tools, task, plan)
        self.debug = debug

    def run(self):
        """
        Execute agent to completion
        """
        turns = 0

        if self.debug:
            for m in self.messages:
                print_message(m)
        
        while turns < 10:
            resp = openai.ChatCompletion.create(model="gpt-4", temperature=0, messages=self.messages)
            assistant_msg = resp.choices[0].message
            self.messages.append(assistant_msg)

            if self.debug:
                print_message(assistant_msg)

            action = extract_action(assistant_msg)
            if not action:
                self.messages.append({ 
                    'role': 'user', 
                    'content': 'Unrecognized action {action} was used. Please use one of the following:\n{tool_description}'.format(action = action.name, tool_description=describe_tools(tools))
                })
            elif action.name == "DONE":
                return
            else:
                tool = resolve_tool(tools, action)
                result = tool()
                self.messages.append({
                    'role': 'user',
                    'content': 'RESULT: {}'.format(result)
                })
            
            
            turns += 1

            

In [ ]:
agent = Agent(tools, task, the_plan, debug=True)

In [ ]:
agent.run()

# Github v2

Support:
- open

In [130]:
import tinygen

from importlib import reload
reload(tinygen)

gh_api_key = os.getenv("GITHUB_API_KEY")
repo_spec = 'https://github.com/leoshimo/copier_python_template'
gh = tinygen.github.GithubRepo(repo_spec, gh_api_key)

In [122]:
# Open
gh.open("/src/../README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [148]:
# Find File
gh.find_file("READ")

['README.md']

In [124]:
gh.search("read")

['README.md', 'pyproject.toml.jinja']

# Workspace V2

Place on `write` calls to store old contents.
Reads are "pass-through" and cached. May be overwritten by writes.
Backed by REPO calls.

In [189]:
reload(tinygen.workspace)
wkspc = tinygen.workspace.Workspace(gh)

In [190]:
wkspc.find("READ")

['README.md']

In [191]:
wkspc.search("READ")

['README.md', 'pyproject.toml.jinja']

In [192]:
wkspc.open("README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [195]:
wkspc.write("README.md", "Hello world!")

In [197]:
print(wkspc.diff())

--- a/README.md
+++ b/README.md
@@ -1,37 +1 @@
-# README
-
-[Copier](https://copier.readthedocs.io/) template for a simple Python projects
-
-- Python project with `src` and `test` managed by `poetry`
-- `notebooks` for Jupyter notebooks
-- `Makefile` as task runner. See `make help`
-
-## Prerequisites
-
-- `pyenv`
-- `poetry`
-- `copier`
-
-## Project Setup
-
-```sh
-$ copier copy gh:leoshimo/copier_python_template my_project
-$ cd my_project
-$ make setup
-```
-
-## Running Tasks
-
-See `make help` for list of tasks.
-
-## API Keys
-
-Add `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:
-
-```python
-import os
-from dotenv import load_dotenv
-load_dotenv()
-os.environ.get('SOME_API_KEY')
-```
-
+Hello world!



# Github Tools